# Pipeline Optimization

# Search over Pipeline-land...

TBD

# Approaches

### Bayesian Optimization

### Genetic Programming

### Meta-learning

### Multi-armed bandits

# Bayesian Optimization

Can we use the same approach that we use for Hyper-parameter Optimization to search for the best ML pipeline?

### Problem
Parameter space becomes huge and we need to handle *conditional parameters*, parameters that are only relevant when others are set in a specific way (e.g. `svm__kernel` only relevant if `classifier=svm`).
This has been refered to as the CASH problem: *Combined Algorithm Selection and Hyper-parameter optimization*.



# Genetic Programming

GP is a evolutionary computation technique for automatically constructing computer programs.

GP evolves computer programs, traditionally represented in memory as tree structures. Every tree node has an operator function and every terminal node has an operand, making mathematical expressions easy to evolve and evaluate (e.g. for symbolic regression).

[Algorithm](http://www.genetic-programming.com/gpanimatedtutorial.html):

  1. Randomly create an initial population (generation 0)
  2. Iteratively perform the following sub-steps (called a generation):
    1. Execute each program in the population and ascertain its fitness
    2. Select programs from the population with a proba based on fitness to participate in the genetic operations in C.
    3. Create new programs for the population by applying the following genetic operations with specified probabilities:
      1. Reproduction: Copy the selected individual program to the new population.
      2. Crossover: Create new offspring program(s) for the new population by recombining randomly chosen parts from two selected programs.
      3. Mutation: Create one new offspring program for the new population by randomly mutating a randomly chosen part of one selected program.
  

# Mutation

<img src="img/gp-mutation.png" >

# Cross-over

<img src="img/gp-cross-over.png" >


# Meta-Learning

Learn from the performance of ML piplines from **different** datasets to predict which ML pipelines will likely do well on the dataset at hand.

Can we used to create a seed set for other techniques (e.g. [auto-sklearn](https://www.automl.org/automl/auto-sklearn/) uses Meta-Learning and Bayesian Optimization). 


Meta-learning model: target is performance and runtime; features are characteristics of the dataset, pipeline, and problem formulation.

Training data: [OpenML](https://www.openml.org/) . 

# Multi-armed Bandits

### Idea
Sample randomly N pipelines, each of them represents an *arm*. We want to select the arm with the highest yield (ie best performance) by spending the least *budget* in total. 

Budget can be iterations (for anytime algorithms), data points (!), or features (!).

Can be seen as an application of early-stopping to pipeline optimization.

<img src="img/map-sh.gif" width=500 >

# Case-study: [TPOT](http://epistasislab.github.io/tpot/)

AML tool that optimizes machine learning pipelines using *genetic programming*.

<img src="img/tpot-logo.jpg" width="200" style="float: right;">

Developed by Randal S. Olson and others at the University of Pennsylvania.

<img src="img/tpot-ml-pipeline.png"  width="600" style="float: left;">


In [1]:
from tpot import TPOTRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

housing = load_boston()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target,
                                                    train_size=0.75, test_size=0.25)

In [2]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

/home/peter/anaconda3/envs/aml/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Generation 1 - Current best internal CV score: -15.341410475898769
Generation 2 - Current best internal CV score: -15.013889792264782
Generation 3 - Current best internal CV score: -14.706598258778271
Generation 4 - Current best internal CV score: -14.706598258778271
Generation 5 - Current best internal CV score: -13.721238360793881

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=True, max_features=0.6500000000000001, min_samples_leaf=1, min_samples_split=5, n_estimators=100)
-7.4721440763637945


In [3]:
tpot.export('tpot_boston_pipeline.py')

In [ ]:
# %load tpot_boston_pipeline.py
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=None)

# Average CV score on the training set was:-13.721238360793881
exported_pipeline = ExtraTreesRegressor(bootstrap=True, max_features=0.6500000000000001, min_samples_leaf=1, min_samples_split=5, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
